In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
import numpy as np

from tensorflow_model_optimization.quantization.keras import QuantizeConfig, quantize_model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import glob

import numpy as np
import tensorflow as tf
from PIL import Image

2023-06-02 13:05:49.086889: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 13:05:54.710474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# lr scheduler
class CosineDecayWithWarmup(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, learning_rate, total_steps, warmup_steps=0):
        super(CosineDecayWithWarmup, self).__init__()
        assert warmup_steps < total_steps
        self.learning_rate = learning_rate
        self.total_steps = total_steps
        self.warmup_steps = warmup_steps
        self.cosine_decay = tf.keras.experimental.CosineDecay(
            learning_rate, total_steps - warmup_steps)

        
    @tf.function
    def __call__(self, step):
        if step < self.warmup_steps:
            return (self.learning_rate / 
                    tf.cast(self.warmup_steps, tf.float32) * 
                    tf.cast((step + 1), tf.float32))
        return self.cosine_decay(step - self.warmup_steps)

    def get_config(self):
        return {"learning_rate": np.array(self.learning_rate),
                "total_steps": np.array(self.total_steps),
                "warmup_steps": np.array(self.warmup_steps)}


In [3]:
from tensorflow.keras.metrics import AUC

class AUROC(AUC):
    def __init__(self, name='auroc', **kwargs):
        super().__init__(name=name, **kwargs)

class AUROC_convert(AUC):
    def __init__(self, name='auroc', **kwargs):
        super().__init__(name=name, **kwargs)

def auroc(y_true, y_pred):
    return AUC(curve='ROC')(y_true, y_pred)

model = keras.models.load_model(
    "/home/sangsooim/1_AAI/project/EfficientNetV2B0-best_model.h5",
    custom_objects={'CosineDecayWithWarmup': CosineDecayWithWarmup, 'AUROC' : AUROC }    
)

print(model)

2023-06-02 13:06:09.839373: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22262 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:c1:00.0, compute capability: 8.6


## make train_images : list

In [4]:
# make train_images
train_images = []
for i in range(1, 49):
    img_path = f"/home/sangsooim/1_AAI/project/practice/frontal/view1_frontal ({i}).jpg"
    img = image.load_img(img_path, target_size=(384, 384))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0
    train_images.append(x)

# Convert the list to a numpy array
# (number_of_images, height, width, channels). (48, 384, 384, 3).
train_images = np.concatenate(train_images, axis=0) 
print(train_images.shape)

(48, 384, 384, 3)


# PTQ

In [5]:
#PTQ
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(48):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 옵션들
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen # 함수로 설정해야 한다.
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8] # 이 모델은 integer 연산만 할 것
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8 # or tf.int8
converter.inference_output_type = tf.uint8 # or tf.int8
# 옵션 끝

post_quant_tflite_model = converter.convert()

2023-06-02 13:06:28.787296: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,96,96,32]
	 [[{{node inputs}}]]
2023-06-02 13:06:28.847695: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,48,48,48]
	 [[{{node inputs}}]]
2023-06-02 13:06:28.889106: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,192]
	 [[{{node inputs}}]]
2023-0

INFO:tensorflow:Assets written to: /tmp/tmpt4r_iz0_/assets


INFO:tensorflow:Assets written to: /tmp/tmpt4r_iz0_/assets
/home/sangsooim/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-06-02 13:07:51.428545: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-02 13:07:51.428668: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-02 13:07:51.441164: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpt4r_iz0_
2023-06-02 13:07:51.626870: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-02 13:07:51.627005: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpt4r_iz0_
2023-06-02 13:07:52.113396: I tensorflow/compiler/ml

In [6]:
with tf.io.gfile.GFile('quant_model.tflite', 'wb') as f:
    f.write(post_quant_tflite_model)